In [1]:
import os 
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from includes.functions import *
import pandas as pd
import json
import os
os.chdir(r'./')

In [2]:
corpus = pd.read_excel("CORPUS/Selected_Articles.xlsx")
with open('CORPUS/annotated_corpus_W2.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

sample = data
sample = [tuple(i) for i in sample]
for i in sample:
    if len(i[1]['entities']) != 0:
      for j in range(len(i[1]['entities'])):
        i[1]['entities'][j] = tuple(i[1]['entities'][j])

In [3]:
#!pip install gliner -q

In [4]:
model_inference = GLiNER.from_pretrained("urchade/gliner_multi-v2.1", max_length=512)
device="cuda"
model_inference.to(device)

labels = ["Magasin de producteurs", "Date", "Région", "Commune", "Département"]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

C:\Users\pthiam\Documents\irit_environement\env\Lib\site-packages\transformers\convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
from includes.DynamicSegmentation import ThematicSegmenter

window_size = 5
#model_name = "camembert-base"
#model_name = "paraphrase-multilingual-MiniLM-L12-v2" # Model By default

segmenter = ThematicSegmenter(
    #model_name = model_name,
    window_size= window_size
)

In [6]:
thematic_seg = split_thematic_segment(corpus, segmenter)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
thematic_seg

[["Ce ne sont ni des Côtes du Rhône, ni des côteaux du Lyonnais. Quatre producteurs de Tartaras, Saint-Jean-de-Toulas, Saint-Maurice-sur-Dargoire et Saint-Joseph, confectionnent des vins de pays des Collines Rhodaniennes. M. Deplaude, viticulteur sur Tartaras (Loire), a 1,2 hectare en exploitation. La vigne n'est qu'une petite partie de son activité. Mais, elle a toujours été présente dans cette région.",
  'Et le viticulteur d\'expliquer : "Au début du siècle, il y avait cinquante hectares de planter dans cette région. Nous vendions essentiellement dans la vallée du Gier où il y avait une grosse consommation de la part des mineurs". Le terroir, composé de schiste assez léger, est idéal pour la culture de la vigne. Le domaine Deplaude produit un vin rouge composé à 80 % de gamay et 20 % de pinot. Mais d\'autres viticulteurs ont également de la syrah. Le blanc est plus rare mais le viticulteur de Tartaras en produit en mélangeant les cépages chardonnay et gamay. Ce vignoble, situé à ext

In [8]:
from includes.SemanticQueryV1 import *
import pandas as pd

concepts = {
    'Magasin de producteurs': "Le magasin de producteurs est un point de vente collectif qui regroupe plusieurs agriculteurs qui ont pour objectif de valoriser leur exploitation et leur production fermière",
}

config = SearchConfig(
    threshold_base = 0.2,
    keyword_weight=0.2,
    semantic_weight=0.8,
    batch_size=16,
    max_sequence_length=512
)

search_engine = SemanticSearchEngine("paraphrase-multilingual-MiniLM-L12-v2", config)
query = concepts['Magasin de producteurs']

results = search_engine.search(
    query=query,
    documents=thematic_seg
)

INFO:includes.SemanticQueryV1:Début de la recherche avec 44 segments issus de 20 documents
Calcul des embeddings:   0%|                                                                     | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Calcul des embeddings:   0%|                                                                     | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Calcul des embeddings:  33%|████████████████████▎                                        | 1/3 [00:00<00:00,  6.52it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Calcul des embeddings:  67%|████████████████████████████████████████▋                    | 2/3 [00:00<00:00,  8.02it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Calcul des embeddings: 100%|█████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.92it/s]
INFO:includes.SemanticQueryV1:Recherche terminée: 36 résultats retournés


In [9]:
results = evaluate_gliner_with_search_results(sample, results, labels, model_inference)
metrics = calculate_metrics(results)
print_metrics(metrics)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Document 0: Segments trouvés: 2/2 (100.0%)
Document 1: Segments trouvés: 2/2 (100.0%)
Document 2: Segments trouvés: 1/1 (100.0%)
Aucun segment trouvé pour le document 3
Document 4: Segments trouvés: 4/4 (100.0%)
Document 5: Segments trouvés: 1/1 (100.0%)
Document 6: Segments trouvés: 2/2 (100.0%)
Document 7: Segments trouvés: 2/2 (100.0%)
Document 8: Segments trouvés: 2/2 (100.0%)
Document 9: Segments trouvés: 4/4 (100.0%)
Document 10: Segments trouvés: 2/2 (100.0%)
Aucun segment trouvé pour le document 11
Document 12: Segments trouvés: 3/3 (100.0%)
Document 13: Segments trouvés: 2/2 (100.0%)
Document 14: Segments trouvés: 1/1 (100.0%)
Document 15: Segments trouvés: 2/2 (100.0%)
Document 16: Segments trouvés: 2/2 (100.0%)
Document 17: Segments trouvés: 2/2 (100.0%)
Document 18: Segments trouvés: 1/1 (100.0%)
Document 19: Segments trouvés: 1/1 (100.0%)

Métriques par type d'entité :
Type                  Précision     Rappel         F1    Support
----------------------------------------

In [10]:
def split_sliding_window(sample , stride = 512):
    tokenizer = create_tokenizer()
    res = []
    for i in range(len(sample)):
        text_segmented = split_text_into_sliding_windows(sample[i][0], tokenizer, max_tokens=512, stride = stride)
        res.append(text_segmented)
    
    return res

sliding_segment_overlap = split_sliding_window(sample, stride = 153) # represents 30% of overlapping

C:\Users\pthiam\Documents\irit_environement\env\Lib\site-packages\transformers\convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [11]:
results = evaluate_gliner_segments(sample, sliding_segment_overlap, labels, model_inference)
metrics = calculate_metrics(results)
print_metrics(metrics)

Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 2/2 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 5/5 (100.0%)
Segments trouvés: 2/2 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 3/3 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 6/6 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 5/5 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)

Métriques par type d'entité :
Type                  Précision     Rappel         F1    Support
--------------------------------------------------------------------------------
Commune                   0.386      0.607      0.472         28
Département               0.250      0.083      0.125         12
Date                      0.739      0.386      0.507         44
Région  

In [12]:
sliding_segment = split_sliding_window(sample)

In [13]:
results = evaluate_gliner_segments(sample, sliding_segment, labels, model_inference)
metrics = calculate_metrics(results)
print_metrics(metrics)

Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 2/2 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 2/2 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 2/2 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)
Segments trouvés: 1/1 (100.0%)

Métriques par type d'entité :
Type                  Précision     Rappel         F1    Support
--------------------------------------------------------------------------------
Commune                   0.447      0.607      0.515         28
Département               0.500      0.083      0.143         12
Date                      0.714      0.341      0.462         44
Région  